This file, once complete, will obtain a dataframe that contains every trade that occured during the 2019-20 NHL season. I am going to scrape the CapFriendly Website using beautiful soup and selenium and then use pandas to get it into the dataframe.

In [1]:
import requests
import pandas as pd
import time
import logging
import lxml
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Now that necessary packages are installed, going to parse the needed url. This URL has a dynamic table that contains every trade that occured in the league for the 2009-10 season. So I am going to scrape the cap friendly website (Trades tab) and get this into a dataframe that I can eventually download and put into the MySQL database. As mentioned, since the table is dynamic and not static, the selenium package is needed so that after the load more button is clicked and no more rows appear (once the page is opened in the selenium server), the table is then downloaded to a dataframe.

In [2]:
# Set up the Selenium driver (make sure you have the correct path to your WebDriver)
driver_path = 'C:/STL Blues Analytics/SalaryCap/NHLSalaryCap/chromedriver.exe'  # Update this to your WebDriver path
service= Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the webpage
url = "https://www.capfriendly.com/trades?season=2010"
driver.get(url)

# Wait for the page to load and click "Load More" until all rows are loaded
while True:
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            #EC.element_to_be_clickable((By.XPATH, "//button[text()='Load More']"))
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[10]/div/div/div[21]/div[1]"))
        )
        load_more_button.click()
        time.sleep(5)  # wait for the new rows to load
        # Check if "No results found" text is present
        no_results_text = driver.find_elements(By.XPATH, "/html/body/div[10]/div/div/div[20]/table/tbody/tr[154]/td/div")
        if no_results_text:
            break  # if "No results found" text is present, exit the loop
    except:
        break  # if no more "Load More" button, exit the loop

# Get the page source after all rows are loaded
page_source = driver.page_source
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table and convert it to a DataFrame
table = soup.find('table')
dfattempt = pd.read_html(str(table))[0]


C:\Users\nsofi\AppData\Local\Temp\ipykernel_6484\1247284739.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfattempt = pd.read_html(str(table))[0]


In [3]:
#print(dfattempt)
dfattempt.to_csv("Sel2010Att1.csv")

In [4]:
dfattempt.shape
dfattempt.tail(5)

,DATE,TRADE DETAILS,TRADE DETAILS.1,TRADE DETAILS.2,TRADE DETAILS.3,TRADE DETAILS.4,TRADE DETAILS.5,TRADE DETAILS.6,TRADE DETAILS.7,TRADE DETAILS.8,...,TRADE DETAILS.24,TRADE DETAILS.25,TRADE DETAILS.26,TRADE DETAILS.27,TRADE DETAILS.28,TRADE DETAILS.29,TRADE DETAILS.30,TRADE DETAILS.31,TRADE DETAILS.32,TRADE DETAILS.33
121,"Jul. 9, 2009",Edmonton Oilers Acquire:2010 Conditional Draft...,Edmonton Oilers Acquire:2010 Conditional Draft...,Edmonton Oilers Acquire:2010 Conditional Draft...,Edmonton Oilers Acquire:2010 Conditional Draft...,Edmonton Oilers Acquire:2010 Conditional Draft...,Edmonton Oilers Acquire:2010 Conditional Draft...,Edmonton Oilers Acquire:2010 Conditional Draft...,Edmonton Oilers Acquire:2010 Conditional Draft...,Edmonton Oilers Acquire:2010 Conditional Draft...,...,New Jersey Devils Acquire:Tim Sestito,New Jersey Devils Acquire:Tim Sestito,New Jersey Devils Acquire:Tim Sestito,New Jersey Devils Acquire:Tim Sestito,New Jersey Devils Acquire:Tim Sestito,New Jersey Devils Acquire:Tim Sestito,New Jersey Devils Acquire:Tim Sestito,New Jersey Devils Acquire:Tim Sestito,New Jersey Devils Acquire:Tim Sestito,New Jersey Devils Acquire:Tim Sestito
122,"Jul. 8, 2009",Dallas Stars Acquire:Alex Auld,Dallas Stars Acquire:Alex Auld,Dallas Stars Acquire:Alex Auld,Dallas Stars Acquire:Alex Auld,Dallas Stars Acquire:Alex Auld,Dallas Stars Acquire:Alex Auld,Dallas Stars Acquire:Alex Auld,Dallas Stars Acquire:Alex Auld,Dallas Stars Acquire:Alex Auld,...,Ottawa Senators Acquire:2010 6th round pick (S...,Ottawa Senators Acquire:2010 6th round pick (S...,Ottawa Senators Acquire:2010 6th round pick (S...,Ottawa Senators Acquire:2010 6th round pick (S...,Ottawa Senators Acquire:2010 6th round pick (S...,Ottawa Senators Acquire:2010 6th round pick (S...,Ottawa Senators Acquire:2010 6th round pick (S...,Ottawa Senators Acquire:2010 6th round pick (S...,Ottawa Senators Acquire:2010 6th round pick (S...,Ottawa Senators Acquire:2010 6th round pick (S...
123,"Jul. 3, 2009",Colorado Avalanche Acquire:Kyle QuinceyTom Pre...,Colorado Avalanche Acquire:Kyle QuinceyTom Pre...,Colorado Avalanche Acquire:Kyle QuinceyTom Pre...,Colorado Avalanche Acquire:Kyle QuinceyTom Pre...,Colorado Avalanche Acquire:Kyle QuinceyTom Pre...,Colorado Avalanche Acquire:Kyle QuinceyTom Pre...,Colorado Avalanche Acquire:Kyle QuinceyTom Pre...,Colorado Avalanche Acquire:Kyle QuinceyTom Pre...,Colorado Avalanche Acquire:Kyle QuinceyTom Pre...,...,Los Angeles Kings Acquire:Ryan Smyth,Los Angeles Kings Acquire:Ryan Smyth,Los Angeles Kings Acquire:Ryan Smyth,Los Angeles Kings Acquire:Ryan Smyth,Los Angeles Kings Acquire:Ryan Smyth,Los Angeles Kings Acquire:Ryan Smyth,Los Angeles Kings Acquire:Ryan Smyth,Los Angeles Kings Acquire:Ryan Smyth,Los Angeles Kings Acquire:Ryan Smyth,Los Angeles Kings Acquire:Ryan Smyth
124,"Jul. 1, 2009",Toronto Maple Leafs Acquire:Colin StuartGarnet...,Toronto Maple Leafs Acquire:Colin StuartGarnet...,Toronto Maple Leafs Acquire:Colin StuartGarnet...,Toronto Maple Leafs Acquire:Colin StuartGarnet...,Toronto Maple Leafs Acquire:Colin StuartGarnet...,Toronto Maple Leafs Acquire:Colin StuartGarnet...,Toronto Maple Leafs Acquire:Colin StuartGarnet...,Toronto Maple Leafs Acquire:Colin StuartGarnet...,Toronto Maple Leafs Acquire:Colin StuartGarnet...,...,Atlanta Thrashers Acquire:Pavel KubinaTim Stap...,Atlanta Thrashers Acquire:Pavel KubinaTim Stap...,Atlanta Thrashers Acquire:Pavel KubinaTim Stap...,Atlanta Thrashers Acquire:Pavel KubinaTim Stap...,Atlanta Thrashers Acquire:Pavel KubinaTim Stap...,Atlanta Thrashers Acquire:Pavel KubinaTim Stap...,Atlanta Thrashers Acquire:Pavel KubinaTim Stap...,Atlanta Thrashers Acquire:Pavel KubinaTim Stap...,Atlanta Thrashers Acquire:Pavel KubinaTim Stap...,Atlanta Thrashers Acquire:Pavel KubinaTim Stap...
125,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,...,No Results Found,No Results F

BOOM!! Scraped all 79 rows I need dynamically using the selenium driver. Now do the data cleaning that I have worked on and come close to getting fully corrected on it. From there, make the last fixes manually. About 90% of getting a cleaned trades dataframe is automated with 10% being manual inside the csv file itslef.

In [5]:
dfattempt.head(5)

,DATE,TRADE DETAILS,TRADE DETAILS.1,TRADE DETAILS.2,TRADE DETAILS.3,TRADE DETAILS.4,TRADE DETAILS.5,TRADE DETAILS.6,TRADE DETAILS.7,TRADE DETAILS.8,...,TRADE DETAILS.24,TRADE DETAILS.25,TRADE DETAILS.26,TRADE DETAILS.27,TRADE DETAILS.28,TRADE DETAILS.29,TRADE DETAILS.30,TRADE DETAILS.31,TRADE DETAILS.32,TRADE DETAILS.33
0,"Jun. 30, 2010",Anaheim Ducks Acquire:Jason Jaffray,Anaheim Ducks Acquire:Jason Jaffray,Anaheim Ducks Acquire:Jason Jaffray,Anaheim Ducks Acquire:Jason Jaffray,Anaheim Ducks Acquire:Jason Jaffray,Anaheim Ducks Acquire:Jason Jaffray,Anaheim Ducks Acquire:Jason Jaffray,Anaheim Ducks Acquire:Jason Jaffray,Anaheim Ducks Acquire:Jason Jaffray,...,Calgary Flames Acquire:Logan MacMillan,Calgary Flames Acquire:Logan MacMillan,Calgary Flames Acquire:Logan MacMillan,Calgary Flames Acquire:Logan MacMillan,Calgary Flames Acquire:Logan MacMillan,Calgary Flames Acquire:Logan MacMillan,Calgary Flames Acquire:Logan MacMillan,Calgary Flames Acquire:Logan MacMillan,Calgary Flames Acquire:Logan MacMillan,Calgary Flames Acquire:Logan MacMillan
1,"Jun. 30, 2010",Chicago Blackhawks Acquire:Philippe ParadisVik...,Chicago Blackhawks Acquire:Philippe ParadisVik...,Chicago Blackhawks Acquire:Philippe ParadisVik...,Chicago Blackhawks Acquire:Philippe ParadisVik...,Chicago Blackhawks Acquire:Philippe ParadisVik...,Chicago Blackhawks Acquire:Philippe ParadisVik...,Chicago Blackhawks Acquire:Philippe ParadisVik...,Chicago Blackhawks Acquire:Philippe ParadisVik...,Chicago Blackhawks Acquire:Philippe ParadisVik...,...,Toronto Maple Leafs Acquire:Kris VersteegBill ...,Toronto Maple Leafs Acquire:Kris VersteegBill ...,Toronto Maple Leafs Acquire:Kris VersteegBill ...,Toronto Maple Leafs Acquire:Kris VersteegBill ...,Toronto Maple Leafs Acquire:Kris VersteegBill ...,Toronto Maple Leafs Acquire:Kris VersteegBill ...,Toronto Maple Leafs Acquire:Kris VersteegBill ...,Toronto Maple Leafs Acquire:Kris VersteegBill ...,Toronto Maple Leafs Acquire:Kris VersteegBill ...,Toronto Maple Leafs Acquire:Kris VersteegBill ...
2,"Jun. 30, 2010",Arizona Coyotes Acquire:Patrick O'Sullivan,Arizona Coyotes Acquire:Patrick O'Sullivan,Arizona Coyotes Acquire:Patrick O'Sullivan,Arizona Coyotes Acquire:Patrick O'Sullivan,Arizona Coyotes Acquire:Patrick O'Sullivan,Arizona Coyotes Acquire:Patrick O'Sullivan,Arizona Coyotes Acquire:Patrick O'Sullivan,Arizona Coyotes Acquire:Patrick O'Sullivan,Arizona Coyotes Acquire:Patrick O'Sullivan,...,Edmonton Oilers Acquire:Jim Vandermeer,Edmonton Oilers Acquire:Jim Vandermeer,Edmonton Oilers Acquire:Jim Vandermeer,Edmonton Oilers Acquire:Jim Vandermeer,Edmonton Oilers Acquire:Jim Vandermeer,Edmonton Oilers Acquire:Jim Vandermeer,Edmonton Oilers Acquire:Jim Vandermeer,Edmonton Oilers Acquire:Jim Vandermeer,Edmonton Oilers Acquire:Jim Vandermeer,Edmonton Oilers Acquire:Jim Vandermeer
3,"Jun. 29, 2010",Montreal Canadiens Acquire:Dan EllisDustin Boyd,Montreal Canadiens Acquire:Dan EllisDustin Boyd,Montreal Canadiens Acquire:Dan EllisDustin Boyd,Montreal Canadiens Acquire:Dan EllisDustin Boyd,Montreal Canadiens Acquire:Dan EllisDustin Boyd,Montreal Canadiens Acquire:Dan EllisDustin Boyd,Montreal Canadiens Acquire:Dan EllisDustin Boyd,Montreal Canadiens Acquire:Dan EllisDustin Boyd,Montreal Canadiens Acquire:Dan EllisDustin Boyd,...,Nashville Predators Acquire:Sergei Kostitsyn,Nashville Predators Acquire:Sergei Kostitsyn,Nashville Predators Acquire:Sergei Kostitsyn,Nashville Predators Acquire:Sergei Kostitsyn,Nashville Predators Acquire:Sergei Kostitsyn,Nashville Predators Acquire:Sergei Kostitsyn,Nashville Predators Acquire:Sergei Kostitsyn,Nashville Predators Acquire:Sergei Kostitsyn,Nashville Predators Acquire:Sergei Kostitsyn,Nashville Predators Acquire:Sergei Kostitsyn
4,"Jun. 28, 2010",Arizona Coyotes Acquire:2012 4th round pick (C...,Arizona Coyotes Acquire:2012 4th round pick (C...,Arizona Coyotes Acquire:2012 4th round pick (C...,Arizona Coyotes Acquire:2012 4th round pick (C...,Arizona Coyotes Acquire:2012 4th round pick (C...

In [6]:
df= pd.read_csv("Sel2010Att1.csv", header=1)
print(df)

     Unnamed: 0           DATE  \
0             0  Jun. 29, 2011   
1             1  Jun. 29, 2011   
2             2  Jun. 29, 2011   
3             3  Jun. 27, 2011   
4             4  Jun. 27, 2011   
..          ...            ...   
106         106  Jul. 19, 2010   
107         107  Jul. 19, 2010   
108         108   Jul. 9, 2010   
109         109   Jul. 1, 2010   
110         110   Jul. 1, 2010   

                                         TRADE DETAILS  \
0    Buffalo Sabres Acquire:2012 7th round pick (FL...   
1       Columbus Blue Jackets Acquire:James Wisniewski   
2             Buffalo Sabres Acquire:Christian Ehrhoff   
3                  Minnesota Wild Acquire:Darroll Powe   
4    Chicago Blackhawks Acquire:2012 7th round pick...   
..                                                 ...   
106               Anaheim Ducks Acquire:Tomas Zaborsky   
107  Philadelphia Flyers Acquire:Matt Walker2011 4t...   
108      Anaheim Ducks Acquire:Aaron VorosRyan Hillier   
109  Chicag

In [7]:
df.head(5)

,Unnamed: 0,DATE,TRADE DETAILS,TRADE DETAILS.1,TRADE DETAILS.2,TRADE DETAILS.3,TRADE DETAILS.4,TRADE DETAILS.5,TRADE DETAILS.6,TRADE DETAILS.7,...,TRADE DETAILS.24,TRADE DETAILS.25,TRADE DETAILS.26,TRADE DETAILS.27,TRADE DETAILS.28,TRADE DETAILS.29,TRADE DETAILS.30,TRADE DETAILS.31,TRADE DETAILS.32,TRADE DETAILS.33
0,0,"Jun. 29, 2011",Buffalo Sabres Acquire:2012 7th round pick (FL...,Buffalo Sabres Acquire:2012 7th round pick (FL...,Buffalo Sabres Acquire:2012 7th round pick (FL...,Buffalo Sabres Acquire:2012 7th round pick (FL...,Buffalo Sabres Acquire:2012 7th round pick (FL...,Buffalo Sabres Acquire:2012 7th round pick (FL...,Buffalo Sabres Acquire:2012 7th round pick (FL...,Buffalo Sabres Acquire:2012 7th round pick (FL...,...,Chicago Blackhawks Acquire:Steve Montador,Chicago Blackhawks Acquire:Steve Montador,Chicago Blackhawks Acquire:Steve Montador,Chicago Blackhawks Acquire:Steve Montador,Chicago Blackhawks Acquire:Steve Montador,Chicago Blackhawks Acquire:Steve Montador,Chicago Blackhawks Acquire:Steve Montador,Chicago Blackhawks Acquire:Steve Montador,Chicago Blackhawks Acquire:Steve Montador,Chicago Blackhawks Acquire:Steve Montador
1,1,"Jun. 29, 2011",Columbus Blue Jackets Acquire:James Wisniewski,Columbus Blue Jackets Acquire:James Wisniewski,Columbus Blue Jackets Acquire:James Wisniewski,Columbus Blue Jackets Acquire:James Wisniewski,Columbus Blue Jackets Acquire:James Wisniewski,Columbus Blue Jackets Acquire:James Wisniewski,Columbus Blue Jackets Acquire:James Wisniewski,Columbus Blue Jackets Acquire:James Wisniewski,...,Montreal Canadiens Acquire:2012 5th round pick...,Montreal Canadiens Acquire:2012 5th round pick...,Montreal Canadiens Acquire:2012 5th round pick...,Montreal Canadiens Acquire:2012 5th round pick...,Montreal Canadiens Acquire:2012 5th round pick...,Montreal Canadiens Acquire:2012 5th round pick...,Montreal Canadiens Acquire:2012 5th round pick...,Montreal Canadiens Acquire:2012 5th round pick...,Montreal Canadiens Acquire:2012 5th round pick...,Montreal Canadiens Acquire:2012 5th round pick...
2,2,"Jun. 29, 2011",Buffalo Sabres Acquire:Christian Ehrhoff,Buffalo Sabres Acquire:Christian Ehrhoff,Buffalo Sabres Acquire:Christian Ehrhoff,Buffalo Sabres Acquire:Christian Ehrhoff,Buffalo Sabres Acquire:Christian Ehrhoff,Buffalo Sabres Acquire:Christian Ehrhoff,Buffalo Sabres Acquire:Christian Ehrhoff,Buffalo Sabres Acquire:Christian Ehrhoff,...,New York Islanders Acquire:2012 4th round pick...,New York Islanders Acquire:2012 4th round pick...,New York Islanders Acquire:2012 4th round pick...,New York Islanders Acquire:2012 4th round pick...,New York Islanders Acquire:2012 4th round pick...,New York Islanders Acquire:2012 4th round pick...,New York Islanders Acquire:2012 4th round pick...,New York Islanders Acquire:2012 4th round pick...,New York Islanders Acquire:2012 4th round pick...,New York Islanders Acquire:2012 4th round pick...
3,3,"Jun. 27, 2011",Minnesota Wild Acquire:Darroll Powe,Minnesota Wild Acquire:Darroll Powe,Minnesota Wild Acquire:Darroll Powe,Minnesota Wild Acquire:Darroll Powe,Minnesota Wild Acquire:Darroll Powe,Minnesota Wild Acquire:Darroll Powe,Minnesota Wild Acquire:Darroll Powe,Minnesota Wild Acquire:Darroll Powe,...,Philadelphia Flyers Acquire:2013 3rd round pic...,Philadelphia Flyers Acquire:2013 3rd round pic...,Philadelphia Flyers Acquire:2013 3rd round pic...,Philadelphia Flyers Acquire:2013 3rd round pic...,Philadelphia Flyers Acquire:2013 3rd round pic...,Philadelphia Flyers Acquire:2013 3rd round pic...,Philadelphia Flyers Acquire:2013 3rd round pic...,Philadelphia Flyers Acquire:2013 3rd round pic...,Philadelphia Flyers Acquire:2013 3rd round pic...,Philadelphia Flyers Acquire:2013 3rd round pic...
4,4,"Jun. 27, 2011",Chicago Blackhawks Acquire:2012 7th round pick...,Chicago Blackhawks Acquire:2012 7th round pick...,Chicago Blackhawks Acquire:2012 7th round pick...,Chicago Blackhawks Acquire:2012 7th round pick...,Chicago Blackhawks Acquire:2012 7th round pick...,Chica

Alright so I can see that everthing I need got extracted into the dataframe which is awesome. All the information I need can get successfully pulled. However, the format is an absolute disaster. I need to filter what information is going into the df and how it is getting pulled. Just going to send to a csv file real quick to view it better.

Alright from looking at it, A ton of information gets repeated (Lot of repeat variables). SO, I just need to filter and then use some text splitting to get information where it needs to go in the dataframe. I am going to attempt to drop the repetive columns of the dataframe in the next code chunk. Since I am dropping all but two columns, I am simply going to make a new dataframe that contains two columns.

In [6]:
better_df= dfattempt[['DATE','TRADE DETAILS', 'TRADE DETAILS.18']]
better_df.head(5)
better_df.to_csv('All Correct Info2009-10.csv')

Now that all the repetitive columns are gone, I am going to split the good trade details column into multiple columns to make the dataframe much cleaner and easier to read. 

In [7]:
better_df[['Team_x', 'x_receieved']] = better_df['TRADE DETAILS'].str.split('Acquire:', expand=True)
better_df[['Team_y', 'y_receieved']] = better_df['TRADE DETAILS.18'].str.split('Acquire:', expand=True)
better_df.head(5)
better_df.to_csv('Close2009-10.csv')

C:\Users\nsofi\AppData\Local\Temp\ipykernel_6484\2414858036.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  better_df[['Team_x', 'x_receieved']] = better_df['TRADE DETAILS'].str.split('Acquire:', expand=True)
C:\Users\nsofi\AppData\Local\Temp\ipykernel_6484\2414858036.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  better_df[['Team_x', 'x_receieved']] = better_df['TRADE DETAILS'].str.split('Acquire:', expand=True)
C:\Users\nsofi\AppData\Local\Temp\ipykernel_6484\2414858036.py:2: SettingWithCopyWarni

In [9]:
#Have to re-read the Close.csv file back in without the last two rows which have nonsense in them. Making the edit straight in csv file and then just re-reading the csv file after
better_df=pd.read_csv("Close2009-10.csv")

In [10]:
# Define the function to split based on conditions
def conditional_splitx(x_receieved):
    if x_receieved.startswith('20') and "[Conditional]" not in x_receieved:
        parts = x_receieved.rsplit(')', 1)
        receivedx = parts[0] + ')'
        Monetary = parts[1] if len(parts) > 1 else ''
    elif x_receieved.startswith('20') and "[Conditional]" in x_receieved:
        parts = x_receieved.rsplit('Sum', 1)
        receivedx = parts[0] 
        Monetary = parts[1] if len(parts) > 1 else ''
    elif "Future Considerations" in x_receieved:
        parts =x_receieved.rsplit('Sum', 1)
        receivedx = parts[0] 
        Monetary = parts[1] if len(parts) > 1 else ''
    else:
        parts = x_receieved.split(' · ', 1)
        receivedx = parts[0]
        Monetary = ' · ' + parts[1] if len(parts) > 1 else ''
    return pd.Series([receivedx, Monetary])

# Apply the function and create new columns
better_df[['receivedx', 'Monetary']] = better_df['x_receieved'].apply(conditional_splitx)
#better_df[['X_received']] = better_df['receivedx'].str.split('Sum',expand=True)




better_df.head(5)
#better_df.to_csv('Progress2022-23.csv')


,Unnamed: 0,DATE,TRADE DETAILS,TRADE DETAILS.18,Team_x,x_receieved,Team_y,y_receieved,receivedx,Monetary
0,0,"Jun. 30, 2010",Anaheim Ducks Acquire:Jason Jaffray,Calgary Flames Acquire:Logan MacMillan,Anaheim Ducks,Jason Jaffray,Calgary Flames,Logan MacMillan,Jason Jaffray,
1,1,"Jun. 30, 2010",Chicago Blackhawks Acquire:Philippe ParadisVik...,Toronto Maple Leafs Acquire:Kris VersteegBill ...,Chicago Blackhawks,Philippe ParadisViktor StalbergChris DiDomenico,Toronto Maple Leafs,Kris VersteegBill Sweatt,Philippe ParadisViktor StalbergChris DiDomenico,
2,2,"Jun. 30, 2010",Arizona Coyotes Acquire:Patrick O'Sullivan,Edmonton Oilers Acquire:Jim Vandermeer,Arizona Coyotes,Patrick O'Sullivan,Edmonton Oilers,Jim Vandermeer,Patrick O'Sullivan,
3,3,"Jun. 29, 2010",Montreal Canadiens Acquire:Dan EllisDustin Boyd,Nashville Predators Acquire:Sergei Kostitsyn,Montreal Canadiens,Dan EllisDustin Boyd,Nashville Predators,Sergei Kostitsyn,Dan EllisDustin Boyd,
4,4,"Jun. 28, 2010",Arizona Coyotes Acquire:2012 4th round pick (C...,Colorado Avalanche Acquire:Daniel Winnik,Arizona Coyotes,2012 4th round pick (COL - #102 - Rhett Holland),Colorado Avalanche,Daniel Winnik,2012 4th round pick (COL - #102 - Rhett Holland),


Just did a bunch of string manipulation to x_received, now have to do it to y_received

In [11]:
# Define the function to split based on conditions
def conditional_splity(y_receieved):
    if y_receieved.startswith('20') and "[Conditional]" not in y_receieved:
        parts = y_receieved.rsplit(')', 1)
        receivedy = parts[0] + ')'
        Monetary2 = parts[1] if len(parts) > 1 else ''
    elif y_receieved.startswith('20') and "[Conditional]" in y_receieved:
        parts =y_receieved.rsplit('Sum', 1)
        receivedy = parts[0] 
        Monetary2 = parts[1] if len(parts) > 1 else ''
    elif "Future Considerations" in y_receieved:
        parts =y_receieved.rsplit('Sum', 1)
        receivedy = parts[0] 
        Monetary2 = parts[1] if len(parts) > 1 else ''
    else:
        parts = y_receieved.split(' · ', 1)
        receivedy = parts[0]
        Monetary2 = ' · ' + parts[1] if len(parts) > 1 else ''
    return pd.Series([receivedy, Monetary2])

# Apply the function and create new columns
better_df[['receivedy', 'Monetary2']] = better_df['y_receieved'].apply(conditional_splity)
#better_df[['X_received']] = better_df['receivedx'].str.split('Sum',expand=True)




better_df.head(5)
#better_df.to_csv('Progress9.csv')

,Unnamed: 0,DATE,TRADE DETAILS,TRADE DETAILS.18,Team_x,x_receieved,Team_y,y_receieved,receivedx,Monetary,receivedy,Monetary2
0,0,"Jun. 30, 2010",Anaheim Ducks Acquire:Jason Jaffray,Calgary Flames Acquire:Logan MacMillan,Anaheim Ducks,Jason Jaffray,Calgary Flames,Logan MacMillan,Jason Jaffray,,Logan MacMillan,
1,1,"Jun. 30, 2010",Chicago Blackhawks Acquire:Philippe ParadisVik...,Toronto Maple Leafs Acquire:Kris VersteegBill ...,Chicago Blackhawks,Philippe ParadisViktor StalbergChris DiDomenico,Toronto Maple Leafs,Kris VersteegBill Sweatt,Philippe ParadisViktor StalbergChris DiDomenico,,Kris VersteegBill Sweatt,
2,2,"Jun. 30, 2010",Arizona Coyotes Acquire:Patrick O'Sullivan,Edmonton Oilers Acquire:Jim Vandermeer,Arizona Coyotes,Patrick O'Sullivan,Edmonton Oilers,Jim Vandermeer,Patrick O'Sullivan,,Jim Vandermeer,
3,3,"Jun. 29, 2010",Montreal Canadiens Acquire:Dan EllisDustin Boyd,Nashville Predators Acquire:Sergei Kostitsyn,Montreal Canadiens,Dan EllisDustin Boyd,Nashville Predators,Sergei Kostitsyn,Dan EllisDustin Boyd,,Sergei Kostitsyn,
4,4,"Jun. 28, 2010",Arizona Coyotes Acquire:2012 4th round pick (C...,Colorado Avalanche Acquire:Daniel Winnik,Arizona Coyotes,2012 4th round pick (COL - #102 - Rhett Holland),Colorado Avalanche,Daniel Winnik,2012 4th round pick (COL - #102 - Rhett Holland),,Daniel Winnik,


Alright ton of progress made, definitely not 100% correct but some manual labor is going to be needed in addition to the web scraping. But, the web scraping saves a ton of time so going to move forward with it for other seasons to build out and help finish the rest of the trade datasets for other seasons. 

To finish the coding aspect of the trades that occured during the 2022-23 season, I am going to delete the extra columns. From there, I will finish this dataset completely by manually editing some of the data in the csv file itslef. 

In [12]:
better_df.head(2)

,Unnamed: 0,DATE,TRADE DETAILS,TRADE DETAILS.18,Team_x,x_receieved,Team_y,y_receieved,receivedx,Monetary,receivedy,Monetary2
0,0,"Jun. 30, 2010",Anaheim Ducks Acquire:Jason Jaffray,Calgary Flames Acquire:Logan MacMillan,Anaheim Ducks,Jason Jaffray,Calgary Flames,Logan MacMillan,Jason Jaffray,,Logan MacMillan,
1,1,"Jun. 30, 2010",Chicago Blackhawks Acquire:Philippe ParadisVik...,Toronto Maple Leafs Acquire:Kris VersteegBill ...,Chicago Blackhawks,Philippe ParadisViktor StalbergChris DiDomenico,Toronto Maple Leafs,Kris VersteegBill Sweatt,Philippe ParadisViktor StalbergChris DiDomenico,,Kris VersteegBill Sweatt,


In [13]:
Final_df= better_df[['DATE','TRADE DETAILS', 'TRADE DETAILS.18','Team_x','receivedx','Team_y','receivedy']]


In [14]:
Final_df.to_csv("NHL 2009-10 Trade Data.csv")

Just a coding note:
To replace part of a string inside a variable of a dataset do the following: 
better_df['part1'] = better_df['part1'].str.replace(':', '')